In [3]:
import pandas as pd
df=pd.read_csv("/Users/lky/Desktop/mlds400-group2/ESD_Dataset_Black_Friday_subset.csv")
df.head(10)

,sku,store,register,trannum,interid,saledate,stype,quantity,orgprice,amt,seq,mic
0,8932772,5004,30,1100,0,2004-11-24,P,1,36.0,9.0,14400019,126
1,8932921,4307,1,1600,0,2004-11-24,P,1,39.0,39.0,868000019,247
2,8932921,4307,1,1600,0,2004-11-24,P,1,39.0,39.0,868100019,247
3,8932921,6204,40,2700,256507198,2004-11-24,P,1,39.0,31.2,867400019,247
4,8932921,6204,40,2700,256507198,2004-11-24,P,1,39.0,31.2,867500019,247
5,8932921,6204,650,1800,256507198,2004-11-24,R,1,39.0,39.0,867600019,247
6,8932921,6204,650,1800,256507198,2004-11-24,R,1,39.0,39.0,867700019,247
7,8932921,7804,680,300,441809140,2004-11-24,P,1,39.0,39.0,867800019,247
8,8932921,7804,680,300,441809140,2004-11-24,P,1,39.0,39.0,867900019,247
9,8932921,8702,840,1900,496705972,2004-11-24,R,1,39.0,39.0,867200019,247


In [5]:
df.shape

(260994, 12)

In [6]:
df.stype.value_counts() 

stype
P    242139
R     18855
Name: count, dtype: int64

In [9]:
#filter the purchase category because we want to find items people want to buy
df[df["stype"] == "P"]

,sku,store,register,trannum,interid,saledate,stype,quantity,orgprice,amt,seq,mic
0,8932772,5004,30,1100,0,2004-11-24,P,1,36.0,9.00,14400019,126
1,8932921,4307,1,1600,0,2004-11-24,P,1,39.0,39.00,868000019,247
2,8932921,4307,1,1600,0,2004-11-24,P,1,39.0,39.00,868100019,247
3,8932921,6204,40,2700,256507198,2004-11-24,P,1,39.0,31.20,867400019,247
4,8932921,6204,40,2700,256507198,2004-11-24,P,1,39.0,31.20,867500019,247
...,...,...,...,...,...,...,...,...,...,...,...,...
260989,8932685,4109,190,3800,0,2004-11-24,P,1,49.0,9.99,426500018,680
260990,8932685,8609,20,3600,559107861,2004-11-24,P,1,49.0,9.99,426400018,680
260991,8932720,9303,500,800,0,2004-11-24,P,1,70.0,46.90,670500018,45
260992,8932720,9603,340,1100,0,2004-11-24,P,1,70.0,46.90,670600018,45


In [10]:
df.info() #there is no null vaue no nothing needs to be done in this table 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260994 entries, 0 to 260993
Data columns (total 12 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   sku       260994 non-null  int64  
 1   store     260994 non-null  int64  
 2   register  260994 non-null  int64  
 3   trannum   260994 non-null  int64  
 4   interid   260994 non-null  int64  
 5   saledate  260994 non-null  object 
 6   stype     260994 non-null  object 
 7   quantity  260994 non-null  int64  
 8   orgprice  260994 non-null  float64
 9   amt       260994 non-null  float64
 10  seq       260994 non-null  int64  
 11  mic       260994 non-null  int64  
dtypes: float64(2), int64(8), object(2)
memory usage: 23.9+ MB


In [17]:
df.describe()

,sku,store,register,trannum,interid,quantity,orgprice,amt,seq,mic
count,2.609940e+05,260994.000000,260994.000000,260994.000000,2.609940e+05,260994.0,260994.00000,260994.000000,2.609940e+05,260994.000000
mean,4.986500e+06,4505.119627,411.705625,2654.844092,1.917784e+08,1.0,36.26482,28.768326,4.939052e+08,453.869897
std,2.847958e+06,2875.373310,253.909920,5193.342483,3.017682e+08,0.0,37.96969,29.588677,2.878815e+08,277.229313
min,5.500000e+01,102.000000,1.000000,100.000000,0.000000e+00,1.0,0.00000,0.010000,7.000000e+00,1.000000
25%,2.617830e+06,2103.000000,200.000000,900.000000,0.000000e+00,1.0,16.00000,12.990000,2.439001e+08,242.000000
50%,4.911193e+06,4109.000000,381.000000,1800.000000,0.000000e+00,1.0,28.00000,20.500000,4.926000e+08,429.000000
75%,7.433770e+06,7204.000000,590.000000,3200.000000,3.482049e+08,1.0,48.00000,37.500000,7.413000e+08,667.000000
max,9.999950e+06,9909.000000,991.000000,99900.000000,9.999015e+08,1.0,795.00000,625.000000,9.999001e+08,999.000000


In [14]:
#explore the only 2 column that has quantitative meanings 
df.quantity.describe()

count    260994.0
mean          1.0
std           0.0
min           1.0
25%           1.0
50%           1.0
75%           1.0
max           1.0
Name: quantity, dtype: float64

In [15]:
df.orgprice.describe()

count    260994.00000
mean         36.26482
std          37.96969
min           0.00000
25%          16.00000
50%          28.00000
75%          48.00000
max         795.00000
Name: orgprice, dtype: float64

In [19]:
df.amt.describe()

count    260994.000000
mean         28.768326
std          29.588677
min           0.010000
25%          12.990000
50%          20.500000
75%          37.500000
max         625.000000
Name: amt, dtype: float64

In [22]:
df[df["orgprice"] == 0][["quantity", "amt", "orgprice"]] 

,quantity,amt,orgprice
119,1,9.00,0.0
120,1,9.00,0.0
355,1,22.31,0.0
373,1,8.50,0.0
1244,1,19.99,0.0
...,...,...,...
260311,1,28.00,0.0
260450,1,7.25,0.0
260574,1,16.00,0.0
260619,1,4.99,0.0


For Next Week: problem to be solved, investigate whether these transanctions make sense by inner join with another table
called skstinfo to find the retail price to see whether we should drop or keep these transaction 